In [1]:
import os
import sys
sys.path.insert(0, '../utils')
sys.path.insert(0, '..')
import logging

logging.basicConfig(format='%(asctime)s - %(message)s', level=logging.INFO)
logger = logging.getLogger('__name__')

import numpy as np
import tensorflow as tf
from tensorflow.keras.metrics import Mean
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import Adam, SGD, Nadam

from modelsTF import *
from loss import *
%reload_ext autoreload
%autoreload 2

In [2]:
# import data
CLEAN_DATA_DIR = '/home/mark/DataBank/PROBA-V-CHKPT/trimmedPatchesDir'
band = 'NIR'
X = np.load(os.path.join(CLEAN_DATA_DIR, f'TRAINpatchesLR_{band}.npy'), allow_pickle=True)
y = np.load(os.path.join(CLEAN_DATA_DIR, f'TRAINpatchesHR_{band}.npy'), allow_pickle=True)

#X = X_train.transpose((0, 3, 4, 2, 1))
#y = y_train.transpose((0, 3, 4, 2, 1)).squeeze(3)
print(f'Input shape: {X.shape} --------> Output shape: {y.shape}')

Input shape: (52971, 32, 32, 9, 1) --------> Output shape: (52971, 96, 96, 1)


In [3]:
X_train, X_val, y_train, y_val, y_train_mask, y_val_mask = train_test_split(
        X, y, y.mask, test_size=0.7, random_state=17)

y = [y_train, y_train_mask]
valData = [X_val, y_val, y_val_mask]

In [4]:
model = WDSRConv3D(scale=3, numFilters=32, kernelSize=(3, 3, 3), numResBlocks=8,
                expRate=8, decayRate=0.8, numImgLR=9, patchSizeLR=32, isGrayScale=True)

2020-02-29 01:38:14,108 - From /home/mark/.local/lib/python3.6/site-packages/tensorflow_addons/layers/wrappers.py:84: Layer.add_variable (from tensorflow.python.keras.engine.base_layer) is deprecated and will be removed in a future version.
Instructions for updating:
Please use `layer.add_weight` method instead.


In [8]:
from trainClass import *
mirrored_strategy = tf.distribute.MirroredStrategy()

In [10]:
with mirrored_strategy.scope():
    model = WDSRConv3D(scale=3, numFilters=32, kernelSize=(3, 3, 3), numResBlocks=8,
                expRate=8, decayRate=0.8, numImgLR=9, patchSizeLR=32, isGrayScale=True)
    l = Losses()
    trainClass = ModelTrainer(model=model,
                           loss=l.shiftCompensatedL1Loss,
                           metric=l.shiftCompensatedcPSNR,
                           optimizer=Nadam(learning_rate=5e-4),
                           ckptDir='home/mark/DataBank/ckpt',
                           logDir='home/mark/DataBank/log')
    trainClass.fitTrainData(X_train, y, 1, 1, 1, valData, 1)

2020-02-29 01:52:43,866 - Unresolved object in checkpoint: (root).optimizer.iter
2020-02-29 01:52:43,867 - Unresolved object in checkpoint: (root).optimizer.beta_1
2020-02-29 01:52:43,867 - Unresolved object in checkpoint: (root).optimizer.beta_2
2020-02-29 01:52:43,867 - Unresolved object in checkpoint: (root).optimizer.decay
2020-02-29 01:52:43,868 - Unresolved object in checkpoint: (root).optimizer.learning_rate
2020-02-29 01:52:43,869 - Unresolved object in checkpoint: (root).optimizer.momentum_cache
2020-02-29 01:52:43,869 - Unresolved object in checkpoint: (root).optimizer's state 'm' for (root).model.layer_with_weights-0.g
2020-02-29 01:52:43,870 - Unresolved object in checkpoint: (root).optimizer's state 'm' for (root).model.layer_with_weights-0.v
2020-02-29 01:52:43,870 - Unresolved object in checkpoint: (root).optimizer's state 'm' for (root).model.layer_with_weights-1.g
2020-02-29 01:52:43,870 - Unresolved object in checkpoint: (root).optimizer's state 'm' for (root).model.l

2020-02-29 01:52:43,899 - Unresolved object in checkpoint: (root).optimizer's state 'm' for (root).model.layer_with_weights-30.g
2020-02-29 01:52:43,900 - Unresolved object in checkpoint: (root).optimizer's state 'm' for (root).model.layer_with_weights-30.v
2020-02-29 01:52:43,900 - Unresolved object in checkpoint: (root).optimizer's state 'm' for (root).model.layer_with_weights-0.layer.bias
2020-02-29 01:52:43,900 - Unresolved object in checkpoint: (root).optimizer's state 'm' for (root).model.layer_with_weights-1.layer.bias
2020-02-29 01:52:43,901 - Unresolved object in checkpoint: (root).optimizer's state 'm' for (root).model.layer_with_weights-2.layer.bias
2020-02-29 01:52:43,901 - Unresolved object in checkpoint: (root).optimizer's state 'm' for (root).model.layer_with_weights-3.layer.bias
2020-02-29 01:52:43,902 - Unresolved object in checkpoint: (root).optimizer's state 'm' for (root).model.layer_with_weights-4.layer.bias
2020-02-29 01:52:43,903 - Unresolved object in checkpoint

2020-02-29 01:52:43,924 - Unresolved object in checkpoint: (root).optimizer's state 'v' for (root).model.layer_with_weights-14.v
2020-02-29 01:52:43,924 - Unresolved object in checkpoint: (root).optimizer's state 'v' for (root).model.layer_with_weights-15.g
2020-02-29 01:52:43,925 - Unresolved object in checkpoint: (root).optimizer's state 'v' for (root).model.layer_with_weights-15.v
2020-02-29 01:52:43,925 - Unresolved object in checkpoint: (root).optimizer's state 'v' for (root).model.layer_with_weights-16.g
2020-02-29 01:52:43,925 - Unresolved object in checkpoint: (root).optimizer's state 'v' for (root).model.layer_with_weights-16.v
2020-02-29 01:52:43,926 - Unresolved object in checkpoint: (root).optimizer's state 'v' for (root).model.layer_with_weights-17.g
2020-02-29 01:52:43,926 - Unresolved object in checkpoint: (root).optimizer's state 'v' for (root).model.layer_with_weights-17.v
2020-02-29 01:52:43,926 - Unresolved object in checkpoint: (root).optimizer's state 'v' for (root

2020-02-29 01:52:43,947 - Unresolved object in checkpoint: (root).optimizer's state 'v' for (root).model.layer_with_weights-29.layer.bias
2020-02-29 01:52:43,947 - Unresolved object in checkpoint: (root).optimizer's state 'v' for (root).model.layer_with_weights-30.layer.bias
2020-02-29 01:52:43,947 - A checkpoint was restored (e.g. tf.train.Checkpoint.restore or tf.keras.Model.load_weights) but not all checkpointed values were used. See above for specific issues. Use expect_partial() on the load status object, e.g. tf.train.Checkpoint.restore(...).expect_partial(), to silence these warnings, or use assert_consumed() to make the check explicit. See https://www.tensorflow.org/alpha/guide/checkpoints#loading_mechanics for details.


Model restored from checkpoint at step 1900.


ValueError: in converted code:

    ../trainClass.py:126 trainStep  *
        gradients = tape.gradient(loss, self.ckpt.model.trainable_variables)
    /home/mark/.local/lib/python3.6/site-packages/tensorflow_core/python/eager/backprop.py:996 gradient
        flat_sources = [_handle_or_self(x) for x in flat_sources]
    /home/mark/.local/lib/python3.6/site-packages/tensorflow_core/python/eager/backprop.py:996 <listcomp>
        flat_sources = [_handle_or_self(x) for x in flat_sources]
    /home/mark/.local/lib/python3.6/site-packages/tensorflow_core/python/eager/backprop.py:687 _handle_or_self
        x = x.handle
    /home/mark/.local/lib/python3.6/site-packages/tensorflow_core/python/distribute/values.py:717 handle
        raise ValueError("`handle` is not available outside the replica context"

    ValueError: `handle` is not available outside the replica context or a `tf.distribute.Strategy.update()` call.


2020-02-29 01:38:46,740 - step 447/15891, loss: 1754.045, psnr: 27.374
2020-02-29 01:38:46,812 - step 448/15891, loss: 2191.771, psnr: 26.344
2020-02-29 01:38:46,875 - step 449/15891, loss: 2304.252, psnr: 25.834
2020-02-29 01:38:46,937 - step 450/15891, loss: 2323.232, psnr: 25.949
2020-02-29 01:38:46,997 - step 451/15891, loss: 2321.422, psnr: 25.998
2020-02-29 01:38:47,055 - step 452/15891, loss: 2233.042, psnr: 26.435
2020-02-29 01:38:47,113 - step 453/15891, loss: 2227.045, psnr: 26.484
2020-02-29 01:38:47,172 - step 454/15891, loss: 2286.730, psnr: 26.317
2020-02-29 01:38:47,228 - step 455/15891, loss: 2350.139, psnr: 26.154
2020-02-29 01:38:47,284 - step 456/15891, loss: 2281.210, psnr: 26.313
2020-02-29 01:38:47,340 - step 457/15891, loss: 2312.568, psnr: 26.226
2020-02-29 01:38:47,397 - step 458/15891, loss: 2357.955, psnr: 26.076
2020-02-29 01:38:47,458 - step 459/15891, loss: 2317.433, psnr: 26.165
2020-02-29 01:38:47,514 - step 460/15891, loss: 2321.454, psnr: 26.165
2020-0

2020-02-29 01:38:59,486 - step 562/15891, loss: 2230.142, psnr: 26.535
2020-02-29 01:38:59,540 - step 563/15891, loss: 2233.198, psnr: 26.525
2020-02-29 01:38:59,593 - step 564/15891, loss: 2241.788, psnr: 26.493
2020-02-29 01:38:59,646 - step 565/15891, loss: 2238.250, psnr: 26.498
2020-02-29 01:38:59,701 - step 566/15891, loss: 2238.435, psnr: 26.498
2020-02-29 01:38:59,757 - step 567/15891, loss: 2240.705, psnr: 26.492
2020-02-29 01:38:59,811 - step 568/15891, loss: 2251.280, psnr: 26.464
2020-02-29 01:38:59,865 - step 569/15891, loss: 2254.864, psnr: 26.450
2020-02-29 01:38:59,919 - step 570/15891, loss: 2258.855, psnr: 26.439
2020-02-29 01:38:59,973 - step 571/15891, loss: 2261.553, psnr: 26.431
2020-02-29 01:39:00,028 - step 572/15891, loss: 2263.393, psnr: 26.427
2020-02-29 01:39:00,082 - step 573/15891, loss: 2271.057, psnr: 26.401
2020-02-29 01:39:00,137 - step 574/15891, loss: 2268.811, psnr: 26.408
2020-02-29 01:39:00,190 - step 575/15891, loss: 2268.437, psnr: 26.407
2020-0

2020-02-29 01:39:07,730 - step 677/15891, loss: 2293.087, psnr: 26.350
2020-02-29 01:39:07,787 - step 678/15891, loss: 2295.583, psnr: 26.341
2020-02-29 01:39:07,844 - step 679/15891, loss: 2293.810, psnr: 26.344
2020-02-29 01:39:07,900 - step 680/15891, loss: 2295.507, psnr: 26.337
2020-02-29 01:39:07,957 - step 681/15891, loss: 2298.676, psnr: 26.326
2020-02-29 01:39:08,014 - step 682/15891, loss: 2302.081, psnr: 26.317
2020-02-29 01:39:08,071 - step 683/15891, loss: 2299.129, psnr: 26.330
2020-02-29 01:39:08,130 - step 684/15891, loss: 2299.017, psnr: 26.329
2020-02-29 01:39:08,185 - step 685/15891, loss: 2296.281, psnr: 26.336
2020-02-29 01:39:08,240 - step 686/15891, loss: 2295.554, psnr: 26.339
2020-02-29 01:39:08,297 - step 687/15891, loss: 2294.636, psnr: 26.339
2020-02-29 01:39:08,350 - step 688/15891, loss: 2297.027, psnr: 26.331
2020-02-29 01:39:08,405 - step 689/15891, loss: 2298.825, psnr: 26.324
2020-02-29 01:39:08,459 - step 690/15891, loss: 2300.092, psnr: 26.320
2020-0

2020-02-29 01:39:15,944 - step 792/15891, loss: 2268.647, psnr: 26.435
2020-02-29 01:39:15,997 - step 793/15891, loss: 2266.581, psnr: 26.443
2020-02-29 01:39:16,051 - step 794/15891, loss: 2265.240, psnr: 26.446
2020-02-29 01:39:16,104 - step 795/15891, loss: 2266.432, psnr: 26.442
2020-02-29 01:39:16,160 - step 796/15891, loss: 2266.766, psnr: 26.440
2020-02-29 01:39:16,213 - step 797/15891, loss: 2267.173, psnr: 26.438
2020-02-29 01:39:16,266 - step 798/15891, loss: 2267.113, psnr: 26.438
2020-02-29 01:39:16,320 - step 799/15891, loss: 2268.530, psnr: 26.433
2020-02-29 01:39:16,373 - step 800/15891, loss: 2267.635, psnr: 26.437
2020-02-29 01:39:18,045 - Validation results... val_loss: 2181.381, val_psnr: 26.788
2020-02-29 01:39:18,102 - step 801/15891, loss: 2266.721, psnr: 26.442
2020-02-29 01:39:18,158 - step 802/15891, loss: 2266.935, psnr: 26.439
2020-02-29 01:39:18,216 - step 803/15891, loss: 2267.164, psnr: 26.438
2020-02-29 01:39:18,272 - step 804/15891, loss: 2272.461, psnr:

2020-02-29 01:39:25,765 - step 905/15891, loss: 2282.222, psnr: 26.398
2020-02-29 01:39:25,819 - step 906/15891, loss: 2283.219, psnr: 26.395
2020-02-29 01:39:25,873 - step 907/15891, loss: 2284.306, psnr: 26.392
2020-02-29 01:39:25,927 - step 908/15891, loss: 2283.568, psnr: 26.394
2020-02-29 01:39:25,981 - step 909/15891, loss: 2282.168, psnr: 26.397
2020-02-29 01:39:26,034 - step 910/15891, loss: 2282.124, psnr: 26.397
2020-02-29 01:39:26,090 - step 911/15891, loss: 2281.107, psnr: 26.402
2020-02-29 01:39:26,144 - step 912/15891, loss: 2279.999, psnr: 26.404
2020-02-29 01:39:26,203 - step 913/15891, loss: 2277.854, psnr: 26.410
2020-02-29 01:39:26,259 - step 914/15891, loss: 2278.052, psnr: 26.409
2020-02-29 01:39:26,317 - step 915/15891, loss: 2276.240, psnr: 26.418
2020-02-29 01:39:26,376 - step 916/15891, loss: 2275.447, psnr: 26.420
2020-02-29 01:39:26,436 - step 917/15891, loss: 2273.782, psnr: 26.428
2020-02-29 01:39:26,493 - step 918/15891, loss: 2274.481, psnr: 26.426
2020-0

...

2020-02-29 01:40:47,919 - step 2035/15891, loss: 2304.382, psnr: 26.340
2020-02-29 01:40:47,974 - step 2036/15891, loss: 2305.505, psnr: 26.337
2020-02-29 01:40:48,029 - step 2037/15891, loss: 2305.783, psnr: 26.336
2020-02-29 01:40:48,087 - step 2038/15891, loss: 2306.017, psnr: 26.335
2020-02-29 01:40:48,141 - step 2039/15891, loss: 2305.563, psnr: 26.337
2020-02-29 01:40:48,196 - step 2040/15891, loss: 2305.884, psnr: 26.336
2020-02-29 01:40:48,252 - step 2041/15891, loss: 2306.006, psnr: 26.335
2020-02-29 01:40:48,306 - step 2042/15891, loss: 2306.522, psnr: 26.334
2020-02-29 01:40:48,359 - step 2043/15891, loss: 2306.568, psnr: 26.333
2020-02-29 01:40:48,413 - step 2044/15891, loss: 2306.443, psnr: 26.333
2020-02-29 01:40:48,467 - step 2045/15891, loss: 2306.689, psnr: 26.333
2020-02-29 01:40:48,522 - step 2046/15891, loss: 2306.442, psnr: 26.334
2020-02-29 01:40:48,576 - step 2047/15891, loss: 2306.215, psnr: 26.335
2020-02-29 01:40:48,634 - step 2048/15891, loss: 2305.579, psnr:

2020-02-29 01:40:55,865 - step 2148/15891, loss: 2304.292, psnr: 26.342
2020-02-29 01:40:55,919 - step 2149/15891, loss: 2304.545, psnr: 26.341
2020-02-29 01:40:55,974 - step 2150/15891, loss: 2304.960, psnr: 26.341
2020-02-29 01:40:56,030 - step 2151/15891, loss: 2305.121, psnr: 26.340
2020-02-29 01:40:56,085 - step 2152/15891, loss: 2305.188, psnr: 26.340
2020-02-29 01:40:56,144 - step 2153/15891, loss: 2305.085, psnr: 26.341
2020-02-29 01:40:56,200 - step 2154/15891, loss: 2304.772, psnr: 26.341
2020-02-29 01:40:56,259 - step 2155/15891, loss: 2304.770, psnr: 26.341
2020-02-29 01:40:56,318 - step 2156/15891, loss: 2304.719, psnr: 26.341
2020-02-29 01:40:56,377 - step 2157/15891, loss: 2304.707, psnr: 26.341
2020-02-29 01:40:56,432 - step 2158/15891, loss: 2304.747, psnr: 26.340
2020-02-29 01:40:56,488 - step 2159/15891, loss: 2304.409, psnr: 26.341
2020-02-29 01:40:56,549 - step 2160/15891, loss: 2304.862, psnr: 26.340
2020-02-29 01:40:56,606 - step 2161/15891, loss: 2304.782, psnr:

2020-02-29 01:41:03,790 - step 2261/15891, loss: 2306.829, psnr: 26.331
2020-02-29 01:41:03,844 - step 2262/15891, loss: 2307.846, psnr: 26.329
2020-02-29 01:41:03,898 - step 2263/15891, loss: 2307.629, psnr: 26.330
2020-02-29 01:41:03,952 - step 2264/15891, loss: 2307.503, psnr: 26.330
2020-02-29 01:41:04,007 - step 2265/15891, loss: 2307.312, psnr: 26.330
2020-02-29 01:41:04,062 - step 2266/15891, loss: 2307.230, psnr: 26.330
2020-02-29 01:41:04,116 - step 2267/15891, loss: 2307.376, psnr: 26.330
2020-02-29 01:41:04,170 - step 2268/15891, loss: 2307.419, psnr: 26.330
2020-02-29 01:41:04,228 - step 2269/15891, loss: 2307.396, psnr: 26.330
2020-02-29 01:41:04,283 - step 2270/15891, loss: 2308.155, psnr: 26.327
2020-02-29 01:41:04,336 - step 2271/15891, loss: 2307.939, psnr: 26.328
2020-02-29 01:41:04,390 - step 2272/15891, loss: 2308.106, psnr: 26.327
2020-02-29 01:41:04,444 - step 2273/15891, loss: 2308.369, psnr: 26.326
2020-02-29 01:41:04,499 - step 2274/15891, loss: 2308.266, psnr:

2020-02-29 01:41:11,669 - step 2374/15891, loss: 2313.517, psnr: 26.307
2020-02-29 01:41:11,728 - step 2375/15891, loss: 2314.403, psnr: 26.304
2020-02-29 01:41:11,789 - step 2376/15891, loss: 2314.625, psnr: 26.304
2020-02-29 01:41:11,844 - step 2377/15891, loss: 2314.956, psnr: 26.302
2020-02-29 01:41:11,898 - step 2378/15891, loss: 2315.042, psnr: 26.302
2020-02-29 01:41:11,957 - step 2379/15891, loss: 2315.495, psnr: 26.301
2020-02-29 01:41:12,012 - step 2380/15891, loss: 2315.350, psnr: 26.302
2020-02-29 01:41:12,066 - step 2381/15891, loss: 2315.448, psnr: 26.301
2020-02-29 01:41:12,120 - step 2382/15891, loss: 2315.249, psnr: 26.302
2020-02-29 01:41:12,175 - step 2383/15891, loss: 2316.041, psnr: 26.300
2020-02-29 01:41:12,239 - step 2384/15891, loss: 2316.045, psnr: 26.300
2020-02-29 01:41:12,303 - step 2385/15891, loss: 2316.022, psnr: 26.300
2020-02-29 01:41:12,359 - step 2386/15891, loss: 2315.392, psnr: 26.303
2020-02-29 01:41:12,419 - step 2387/15891, loss: 2315.036, psnr:

KeyboardInterrupt: 